<a href="https://colab.research.google.com/github/rahulrajbo/Recommender_system/blob/main/Movies_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries**

In [120]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import ast

from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# **Load Dataset**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
movies = pd.read_csv('/content/drive/MyDrive/Machine Learning/Recommender System/Movies Recommender System/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/drive/MyDrive/Machine Learning/Recommender System/Movies Recommender System/tmdb_5000_credits.csv')

In [22]:
# To display all the columns in the dataset

pd.set_option('display.max_columns', None)

In [23]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [24]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [25]:
print('The shape of movies dataset :', movies.shape)
print('The shape of credits dataset :', credits.shape)

The shape of movies dataset : (4803, 20)
The shape of credits dataset : (4803, 4)


**Merge Both DataFrame**

In [26]:
df = pd.merge(movies, credits, on = 'title')
df.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [19]:
print('The Shape of the merge dataset is :', df.shape)

The Shape of the merge dataset is : (4809, 23)


# **Data Preprocessing**

**Drop Columns which are not too useful**

In [63]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

'budget' is not a criteria to recommend movies.

'homepage' is not a criteria to recommend movies.

'original_language' is not a criteria to recommend movies.

'original_title' is not a criteria to recommend movies.

'production_companies' is not a criteria to recommend movies.

'production_countries' is not a criteria to recommend movies.

'release_date' is not a criteria to recommend movies.

'revenue' is not a criteria to recommend movies.

'runtime' is not a criteria to recommend movies.

'spoken_languages' is not a criteria to recommend movies.

'status' is not a criteria to recommend movies.

'tagline' is not a criteria to recommend movies.

'vote_average' is not a criteria to recommend movies.

'vote_count' is not a criteria to recommend movies.

'Movie_id' is not a criteria to recommend movies.

In [64]:
data = df[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
data.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


**Information about the dataset**

In [65]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   overview  4806 non-null   object
 3   genres    4809 non-null   object
 4   keywords  4809 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 300.6+ KB


**Null Values in the dataset**

In [66]:
data.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

There are three null values for 'overview' feature. So we will drop those  null values.

In [67]:
data.dropna(inplace = True)

In [68]:
# Again check null values

data.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

**Duplicate data in the dataset**

In [69]:
data.duplicated().sum()

0

**Let's check 'genre' feature**

In [70]:
# Let's check 'genre' for 1st movie

data['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

We only want genre for each movie not something like list of dictionaries.

In [71]:
def convert(obj):
  lst = []
  for i in ast.literal_eval(obj):
    lst.append(i['name'])
  return lst

In [72]:
# Apply above function to the 'genre' feature of the dataset. And check genre for 1st movie

data['genres'].apply(convert)[0]

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [73]:
# Apply it in dataset

data['genres'] = data['genres'].apply(convert)

**Let's check 'keywords' feature**

In [74]:
# Let's check 'keywords' for 1st movie

data['keywords'][0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [75]:
# Apply convert function to the 'keywords' feature of the dataset. And check keywords for 1st movie

data['keywords'].apply(convert)[0]

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [76]:
# Apply it in dataset

data['keywords'] = data['keywords'].apply(convert)

**Let's check 'cast' feature**

In [77]:
# Let's check 'genre' for 1st movie

data['cast'][0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [78]:
# Create function to select 1st three characters

def convert_three(obj):
  lst = []
  count = 0
  for i in ast.literal_eval(obj):
    if count !=3:
      lst.append(i['name'])
      count+=1
    else:
      break
  return lst

In [79]:
# Apply function to the 'cast' feature of the dataset to select 1st three characters. And check cast for 1st movie

data['cast'].apply(convert_three)[0]

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']

In [80]:
# Apply it in dataset

data['cast'] = data['cast'].apply(convert_three)

**Let's check 'crew' feature**

In [81]:
# Let's check 'genre' for 1st movie

data['crew'][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [82]:
# Create function to select only 'Directors' for 'crew' feature

def director(obj):
  lst = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      lst.append(i['name'])
      break
  return lst

In [83]:
# Apply function to the 'crew' feature of the dataset to select 1st three crew members. And check crew for 1st movie

data['crew'].apply(director)[0]

['James Cameron']

In [84]:
# Apply it in dataset

data['crew'] = data['crew'].apply(director)

**Let's check 'overview' feature**

In [85]:
# Let's check 'overview' for 1st movie

data['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [86]:
# We are converting the 'overview' feature which is a string into a list

data['overview'].apply(lambda x:x.split())[0]

['In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.']

In [87]:
# Apply it in dataset

data['overview'] = data['overview'].apply(lambda x:x.split())

**Now the correct data**

In [88]:
data.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


We have some space in names in 'genre', 'keywords', 'cast', 'crew' features which will confuse the model.

ex- the name of director of one movie is - rahul singh
and the name of director of other movie is - rahul bora.

now when we recommend based on director 'rahul' then the model get confused which director we talking about.

So, we have to remove the space between names.

In [89]:
data['genres'] = data['genres'].apply(lambda x: [i.replace(' ', '') for i in x])
data['keywords'] = data['keywords'].apply(lambda x: [i.replace(' ', '') for i in x])
data['cast'] = data['cast'].apply(lambda x: [i.replace(' ', '') for i in x])
data['crew'] = data['crew'].apply(lambda x: [i.replace(' ', '') for i in x])

Now we are going to make a new feature in which we are going to concatenate all other features-- 'overview', 'genres', 'keywords', 'cast', 'crew'.

In [90]:
data['tags'] = data['overview'] + data['genres'] + data['keywords'] + data['cast'] + data['crew']
data.head(1)

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [91]:
# Make a new data

new_data = data[['movie_id', 'title', 'tags']]
new_data.head()

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [92]:
# Convert the 'tags' feature which contains information in list into string

new_data['tags'] = new_data['tags'].apply(lambda x: ' '.join(x))
new_data.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [93]:
# Now convert all the data of 'tags' feature in lowercase

new_data['tags'] = new_data['tags'].apply(lambda x: x.lower())
new_data.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


# **Vectorization**

We have to convert  the 'tags' feature in vectors so that we can determine the closest tags or vectors.

For it we are going to use text vectorization --  Bag of Words

In [96]:
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [99]:
vector = cv.fit_transform(new_data['tags']).toarray()
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [98]:
# Shape

cv.fit_transform(new_data['tags']).toarray().shape

(4806, 5000)

In [100]:
# Vector of 1st movie

vector[0]

array([0, 0, 0, ..., 0, 0, 0])

In [105]:
# let's see the 5000 most frequent words

cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [109]:
new_data['tags'][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

In [117]:
# Some words are same but have different prefixes, so we have to remove them by using lemmatizer

lt = WordNetLemmatizer()

def lem(text):
  y = []
  for i in text.split():
    y.append(lt.lemmatize(i))
  return ' '.join(y)

In [118]:
new_data['tags'].apply(lem)[0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following order and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

In [119]:
# Apply to the dataset

new_data['tags'] = new_data['tags'].apply(lem)

**Now we are going to find the distance between different vectors by calculating the cosine distance**

In [121]:
# Again make new vectors by using lemmatize text

cv = CountVectorizer(max_features = 5000, stop_words = 'english')
vector = cv.fit_transform(new_data['tags']).toarray()
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [122]:
cosine_similarity(vector).shape

(4806, 4806)

In [123]:
# Let's find out the similarity score of one movie with all the others movie

similarity = cosine_similarity(vector)

In [125]:
# The highest the similarity value the highest the similarity for recommendation and vice versa

similarity[1]                          # similarity score/ cosine distance of movie 2nd movie with all other movies

array([0.08585457, 1.        , 0.06154575, ..., 0.02512595, 0.        ,
       0.02718636])

In [132]:
# highest similarity value for 2nd movie

sorted(similarity[1], reverse = True)

[1.0000000000000002,
 0.4276686017238498,
 0.2674603192151959,
 0.26057865332352387,
 0.2132007163556105,
 0.21110016546037452,
 0.20751433915982243,
 0.2048366225996757,
 0.2037424855426821,
 0.20032733246124987,
 0.19462473604038075,
 0.19312181983410706,
 0.18759161984421677,
 0.1846372364689991,
 0.18148850216015697,
 0.18148850216015697,
 0.18148850216015694,
 0.17912443020795962,
 0.17407765595569785,
 0.17407765595569785,
 0.17407765595569785,
 0.17232808737106584,
 0.16943474841747155,
 0.1651445647689541,
 0.1651445647689541,
 0.1651445647689541,
 0.16412198797244368,
 0.161627045958809,
 0.1592795619605697,
 0.15745916432444343,
 0.1574591643244434,
 0.15632634987018063,
 0.15569978883230462,
 0.1556997888323046,
 0.15399810070180361,
 0.15386436372416593,
 0.1523510148357077,
 0.1519474352795173,
 0.15151515151515155,
 0.1511417309806357,
 0.15075567228888184,
 0.14999234752441917,
 0.149270358506633,
 0.149270358506633,
 0.1484539238050411,
 0.1484539238050411,
 0.148453923

In [135]:
# But we want high similarity value with their index

sorted(list(enumerate(similarity[1])), reverse= True, key= lambda x: x[1])

[(0, 0.9999999999999998),
 (2409, 0.2666903735313325),
 (507, 0.26601065911288363),
 (539, 0.26155718717359333),
 (1194, 0.24534987303147354),
 (260, 0.24012009007506568),
 (1216, 0.23995690956687135),
 (1444, 0.2372894989381248),
 (582, 0.23485569615051038),
 (3730, 0.2324952774876386),
 (1920, 0.22830583024432843),
 (4048, 0.22430303232870108),
 (1204, 0.22305671869347438),
 (322, 0.21919864974047634),
 (778, 0.21582051197002577),
 (61, 0.2152488010002526),
 (466, 0.2152488010002526),
 (4192, 0.2152488010002526),
 (942, 0.21369687880543226),
 (2333, 0.2135605490443123),
 (74, 0.21356054904431226),
 (83, 0.21137012653545936),
 (495, 0.21029989116983447),
 (3608, 0.20878692266483084),
 (1089, 0.20743642876129637),
 (972, 0.20668850837512243),
 (301, 0.2047597175387335),
 (577, 0.20343336780168372),
 (1201, 0.20134681656420733),
 (47, 0.1979131233319198),
 (1329, 0.19727878476642874),
 (973, 0.19451950503185494),
 (1275, 0.19344776406574832),
 (2204, 0.1917988185229168),
 (2971, 0.19101

# **Main Function to get all movies based on given movie**

In [146]:
# Function to fetch 5 recommend movies based on the given movie

def recommend(movie):
  movie_index = new_data[new_data['title'] == movie].index[0]
  distances = similarity[movie_index]
  movies_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x: x[1])[1:6]

  for i in movies_list:
    print(new_data.iloc[i[0]].title)

In [147]:
# If movie = 'Batman Begins'

recommend('Batman Begins')

The Dark Knight
The Dark Knight Rises
Batman
Batman & Robin
Batman
